In [ ]:
import uproot
import vector
import matplotlib.pyplot as plt
import awkward as ak
import numpy as np
from scipy import stats

tree1 = uproot.open('../Delphes/delphes_output_emu_plus_smeft_cWtil_NP0.root:Delphes')
tree2 = uproot.open('../Delphes/delphes_output_cWtil_real_final.root:Delphes')
tree3 = uproot.open('../Delphes/delphes_output_emu_plus_smeft_cWtil_NP2.root:Delphes')

tree1_cHWtil = uproot.open('../Delphes/delphes_output_cHWtil_real_final.root:Delphes')

events_NP0 = tree1.arrays(['Event.Weight','Muon_size','Muon.PT','Muon.Eta','Muon.Phi', 
                    'Electron_size', 'Electron.PT', 'Electron.Eta','Electron.Phi', 
                    'Jet.PT', 'Jet.Eta', 'Jet.Phi', 'Jet.DeltaEta', 'Jet.DeltaPhi', 'Jet_size', 'Jet.Mass',
                    'Jet.BTag', 'MissingET.MET', 'MissingET_size'])

events_NP1 = tree1_cHWtil.arrays(['Event.Weight','Muon_size','Muon.PT','Muon.Eta','Muon.Phi', 
                    'Electron_size', 'Electron.PT', 'Electron.Eta','Electron.Phi', 
                    'Jet.PT', 'Jet.Eta', 'Jet.Phi', 'Jet.DeltaEta', 'Jet.DeltaPhi', 'Jet_size', 'Jet.Mass',
                    'Jet.BTag', 'MissingET.MET', 'MissingET_size'])

events_NP2 = tree3.arrays(['Event.Weight','Muon_size','Muon.PT','Muon.Eta','Muon.Phi', 
                    'Electron_size', 'Electron.PT', 'Electron.Eta','Electron.Phi', 
                    'Jet.PT', 'Jet.Eta', 'Jet.Phi', 'Jet.DeltaEta', 'Jet.DeltaPhi', 'Jet_size', 'Jet.Mass',
                    'Jet.BTag', 'MissingET.MET', 'MissingET_size'])

N_EVENTS_NP0 = len(events_NP0['Event.Weight'])
N_EVENTS_NP1 = len(events_NP1['Event.Weight'])
N_EVENTS_NP2 = len(events_NP2['Event.Weight'])

def initial_lepton_cuts(events_array):
    
    #lepton selection
    nlepton_mask = (events_array['Muon_size'] == 1) & (events_array['Electron_size'] == 1)
    events_number = np.array([len(events_array['Event.Weight'])])
    muon_eta_mask = (abs(events_array['Muon.Eta']) < 2.5)
    electron_eta_mask1 = (abs(events_array['Electron.Eta']) < 2.47)
    electron_eta_mask2 = (abs(events_array['Electron.Eta']) >= 1.52) | (abs(events_array['Electron.Eta']) <= 1.37)
    muon_pt_mask = (events_array['Muon.PT'] > 27)
    electron_pt_mask = (events_array['Electron.PT'] > 27)
    
    total_muon_mask = muon_pt_mask & muon_eta_mask 
    total_electron_mask = electron_eta_mask1 & electron_eta_mask2 & electron_pt_mask
    
    
    filtered_event_mask = (ak.sum(total_electron_mask, axis=1) == 1) & (ak.sum(total_muon_mask, axis=1) == 1)
    
    
    return filtered_event_mask

def initial_jet_cuts(events_array):
    #jet selection
    events_number = np.array([len(events_array['Event.Weight'])])
    njet_mask = (events_array['Jet_size'] >= 2)
    jet_pt_mask = (events_array['Jet.PT'] > 25)
    jet_eta_mask = (abs(events_array['Jet.Eta']) <= 4.5)
    jet_btag_mask = ((events_array['Jet.PT'] > 25) & (abs(events_array['Jet.Eta']) <= 4.5)) & (events_array['Jet.BTag']==1)
    
    total_jet_mask = jet_pt_mask & jet_eta_mask & ~jet_btag_mask
    

    jet_filtered_mask = ak.sum(total_jet_mask, axis=1) >= 2

    cut_flow = events_array['Event.Weight']
    cut_flow = cut_flow[jet_filtered_mask]
    events_number = np.append(events_number, len(cut_flow))
   
    
    return jet_filtered_mask

def transverse_mass_calc(dilepton_system, met):
    transverse_energy = dilepton_system.et
    transverse_momentum = dilepton_system.pt
    return np.sqrt((transverse_energy + met)**2 - (np.abs(transverse_momentum + met))**2)

def invariant_mass_calc(obj1, obj2):
    invariant_mass  = (obj1 + obj2).mass
    return invariant_mass

def jets_final_cuts(jets):
    #jet_lead_pt_mask = (jets[:,0]).pt > 65
    jet_lead_pt_mask = (jets[:,0]).pt > 20
    #jet_sublead_pt_mask = ((jets[:,1]).pt > 35)
    jet_sublead_pt_mask = (jets[:,1]).pt > 20
    jet_deltaeta_mask = (np.abs(jets[:,0].rapidity - jets[:,1].rapidity) > 2)
    dijet_mass = invariant_mass_calc(jets[:,0], jets[:,1])
    #dijet_mass_mask = (dijet_mass > 100 ) & (dijet_mass < 2000)
    #dijet_mass_mask = (dijet_mass <160)
    dijet_mass_mask = (dijet_mass > 500)
    final_jet_mask = jet_lead_pt_mask & jet_sublead_pt_mask #& dijet_mass_mask & jet_deltaeta_mask
    return final_jet_mask

def leptons_mass_cut(electrons, muons):
    emu_mass = invariant_mass_calc(electrons[:,0], muons[:,0])
    final_dilepton_mass_mask = (emu_mass > 20)
    
    return final_dilepton_mass_mask

def missing_et_cut(missing_et):
    return ak.flatten(missing_et > 30)

def cut_flow_diagram(array):
    bar_labels = ['No cuts', 'Initial lepton cuts', 'Initial Jet cuts', 
                  'Jet invariant mass cuts', 'Dilepton mass cut', 'Missing ET cut']
    y_pos = range(len(bar_labels))
    plt.figure()
    plt.bar(y_pos, array)
    plt.xticks(y_pos, bar_labels, rotation = 'vertical')
    plt.title('Cut Flow Diagram')
    plt.ylabel('Events')
    plt.show()

def apply_cuts(array):
    jets = vector.zip({'pt':array['Jet.PT'],'eta':array['Jet.Eta'],'phi':array['Jet.Phi'], 'mass':array['Jet.Mass']})
    electrons = vector.zip({'pt':array['Electron.PT'],'eta':array['Electron.Eta'],'phi':array['Electron.Phi'],'mass':0.000511})
    muons = vector.zip({'pt':array['Muon.PT'],'eta':array['Muon.Eta'],'phi':array['Muon.Phi'],'mass':0.10566})  
            
    events_number = np.array([len(electrons)])
    initial_lepton_filter = initial_lepton_cuts(array)
    initial_jet_filter = initial_jet_cuts(array)
    missing_et = array['MissingET.MET']
    events_number = np.append(events_number, len(electrons[initial_lepton_filter]))
    electrons_filtered = electrons[initial_lepton_filter & initial_jet_filter]
    jets_filtered = jets[initial_lepton_filter & initial_jet_filter]
    muons_filtered = muons[initial_lepton_filter & initial_jet_filter]
    missing_et_filtered = missing_et[initial_lepton_filter & initial_jet_filter]
    events_number = np.append(events_number, len(electrons_filtered))
    
    #apply jet filter
    total_jet_mask = jets_final_cuts(jets_filtered)
    electrons_filtered = electrons_filtered[total_jet_mask]
    jets_filtered = jets_filtered[total_jet_mask]
    muons_filtered = muons_filtered[total_jet_mask]
    missing_et_filtered = missing_et_filtered[total_jet_mask]
    events_number = np.append(events_number, len(electrons_filtered))
    
    #apply dilepton mass cut
    dilepton_mass_mask = leptons_mass_cut(electrons_filtered, muons_filtered)
    electrons_filtered = electrons_filtered[dilepton_mass_mask]
    jets_filtered = jets_filtered[dilepton_mass_mask]
    muons_filtered = muons_filtered[dilepton_mass_mask]
    missing_et_filtered = missing_et_filtered[dilepton_mass_mask]
    events_number = np.append(events_number, len(electrons_filtered))

    #apply missing ET cut
    missing_et_mask = missing_et_cut(missing_et_filtered)
    jets_filtered = jets_filtered[missing_et_mask]
    muons_filtered = muons_filtered[missing_et_mask]
    missing_et_filtered = missing_et_filtered[missing_et_mask]
    electrons_filtered = electrons_filtered[missing_et_mask]
    events_number = np.append(events_number, len(electrons_filtered))
    
    #apply cuts to weights
    filtered_weights = array['Event.Weight'][initial_lepton_filter & initial_jet_filter]
    filtered_weights = filtered_weights[total_jet_mask]
    filtered_weights = filtered_weights[dilepton_mass_mask]
    filtered_weights = filtered_weights[missing_et_mask]

    return electrons_filtered, muons_filtered, jets_filtered, missing_et_filtered, filtered_weights

def weight_normalisation(array1, weights1, cross_section):
    luminosity = 139 # fb^-1
    normalisation = (1 / np.sum(array1['Event.Weight'])) * luminosity * cross_section
    return normalisation * weights1

def check_cross_section(cross_x, weights):
    return 1/(cross_x/(np.sum(weights)))

def get_dilepton_mass(electron_array, muon_array):
    dilepton_mass = invariant_mass_calc(electron_array[:,0], muon_array[:,0])
    return dilepton_mass

def get_transverse_mass(electron_array, muon_array, missing_et_array):
    dilepton_system_array = electron_array[:,0]+muon_array[:,0]
    transverse_mass = transverse_mass_calc(dilepton_system_array, missing_et_array)
    return transverse_mass

def get_dijet_mass(jet_array):
    dijet_mass = invariant_mass_calc(jet_array[:,0], jet_array[:,1])
    return dijet_mass

def combine_arrays(array1, array2, array3):
    return ak.concatenate([array1, array2, array3], mergebool=True, highlevel=True)

def get_signed_azimuthal_lj(jet_array, electron_array, muon_array, weights):
    
    ordered_pt_mask1 = (electron_array[:,0]).pt > (muon_array[:,0]).pt
    ordered_electrons = electron_array[ordered_pt_mask1]
    ordered_weights1 = weights[ordered_pt_mask1]
    
    ordered_muons = muon_array[~ordered_pt_mask1]
    
    ordered_weights2 = weights[~ordered_pt_mask1]

    lepton_array = ak.concatenate([ordered_electrons, ordered_muons], mergebool=True, highlevel=True)
    weights = ak.concatenate([ordered_weights1, ordered_weights2], mergebool=True, highlevel=True)

    ordered_eta_mask1 = (jet_array[:,0]).eta > (lepton_array[:,0]).eta
    ordered_jets1 = jet_array[ordered_eta_mask1]
    ordered_lepton1 = lepton_array[ordered_eta_mask1]
    ordered_weights1 = weights[ordered_eta_mask1]
    signed_azimuthal_lj1 = ordered_jets1[:,0].phi - ordered_lepton1[:,0].phi

    ordered_jets2 = jet_array[~ordered_eta_mask1]
    ordered_lepton2 = lepton_array[~ordered_eta_mask1]
    ordered_weights2 = weights[~ordered_eta_mask1]
    signed_azimuthal_lj2 = ordered_lepton2[:,0].phi - ordered_jets2[:,0].phi

    ordered_signed_azimuthal_lj = ak.concatenate([signed_azimuthal_lj1, signed_azimuthal_lj2], mergebool=True, highlevel=True)
    weights_total = ak.concatenate([ordered_weights1, ordered_weights2], mergebool=True, highlevel=True)
    ordered_jets = ak.concatenate([ordered_jets1, ordered_jets2], mergebool=True, highlevel=True)

    return ordered_signed_azimuthal_lj, ordered_jets, weights_total

def order_lepton_pt(electron_array,muon_array, weights):
    ordered_pt_mask1 = (electron_array[:,0]).pt > (muon_array[:,0]).pt
    ordered_electrons = electron_array[ordered_pt_mask1]
    ordered_weights1 = weights[ordered_pt_mask1]
    
    ordered_muons = muon_array[~ordered_pt_mask1]
    
    ordered_weights2 = weights[~ordered_pt_mask1]
    
    ordered_lepton_pt = ak.concatenate([(ordered_electrons[:,0]).pt, (ordered_muons[:,0]).pt], mergebool=True, highlevel=True)
    
    ordered_weights = ak.concatenate([ordered_weights1, ordered_weights2], mergebool=True, highlevel=True)
    return ordered_lepton_pt, ordered_weights

def order_signed_azimuthal(jet_array, weights, *args):
    if len(args)>0:
        lepton = args[0]
        jet = args[1]
        ordered_eta_mask1 = (jet_array[:,0]).eta > (lepton[:,0]).eta
        ordered_jets1 = jet[ordered_eta_mask1]
        ordered_lepton1 = lepton[ordered_eta_mask1]
        weights1 = weights[ordered_eta_mask1]
               
        ordered_jets2 = jet[~ordered_eta_mask1]
        ordered_lepton2 = lepton[~ordered_eta_mask1]
        weights2 = weights[~ordered_eta_mask1]
        
    else:
        ordered_eta_mask1 = (jet_array[:,0]).eta > (jet_array[:,1]).eta
        ordered_jets1 = jet_array[ordered_eta_mask1]
        weights1 = weights[ordered_eta_mask1]

        ordered_jets2 = jet_array[~ordered_eta_mask1]
        weights2 = weights[~ordered_eta_mask1]

    ordered_jets = ak.concatenate([ordered_jets1, ordered_jets2], mergebool=True, highlevel=True)
    ordered_weights = ak.concatenate([weights1, weights2], mergebool=True, highlevel=True)
    return ordered_jets, ordered_weights

def get_signed_azimuthal(jet_array, weights, *args):
    
    if len(args)>0:
        lepton = args[0]
        ordered_eta_mask1 = (jet_array[:,0]).eta > (lepton[:,0]).eta
        ordered_jets1 = jet_array[ordered_eta_mask1]
        ordered_lepton1 = lepton[ordered_eta_mask1]
        weights1 = weights[ordered_eta_mask1]
        signed_azimuthal1 = ordered_jets1[:,0].phi - ordered_lepton1[:,0].phi
        
        ordered_jets2 = jet_array[~ordered_eta_mask1]
        ordered_lepton2 = lepton[~ordered_eta_mask1]
        weights2 = weights[~ordered_eta_mask1]
        signed_azimuthal2 = ordered_lepton2[:,0].phi - ordered_jets2[:,0].phi

    else:        
        ordered_eta_mask1 = (jet_array[:,0]).eta > (jet_array[:,1]).eta
        ordered_jets1 = jet_array[ordered_eta_mask1]
        weights1 = weights[ordered_eta_mask1]
        signed_azimuthal1 = ordered_jets1[:,0].phi - ordered_jets1[:,1].phi

        
        ordered_jets2 = jet_array[~ordered_eta_mask1]
        weights2 = weights[~ordered_eta_mask1]
        signed_azimuthal2 = ordered_jets2[:,1].phi - ordered_jets2[:,0].phi
   
    signed_azimuthal = ak.concatenate([signed_azimuthal1, signed_azimuthal2], mergebool=True, highlevel=True)
    
    weights_total = ak.concatenate([weights1, weights2], mergebool=True, highlevel=True)
    return signed_azimuthal, weights_total

def order_signed_azimuthal_overflow(signed_azimuthal, jet_array):
    plus_overflow_mask = signed_azimuthal > np.pi
    plus_overflow_jets = jet_array[plus_overflow_mask]

    minus_overflow_mask = signed_azimuthal < -np.pi
    minus_overflow_jets = jet_array[minus_overflow_mask]

    overflow_mask = plus_overflow_mask | minus_overflow_mask

    not_overflowed_jets = jet_array[~overflow_mask]

    ordered_overflow_jets = ak.concatenate([not_overflowed_jets, plus_overflow_jets, minus_overflow_jets], mergebool=True, highlevel=True)
    return ordered_overflow_jets

def signed_azimuthal_overflow(signed_azimuthal, signed_weights):
            
    plus_overflow_mask = signed_azimuthal > np.pi
    plus_overflow = signed_azimuthal[plus_overflow_mask]
    plus_overflow_weights = signed_weights[plus_overflow_mask]

    minus_overflow_mask = signed_azimuthal < -np.pi
    minus_overflow = signed_azimuthal[minus_overflow_mask]
    minus_overflow_weights = signed_weights[minus_overflow_mask]

    overflow_mask = plus_overflow_mask | minus_overflow_mask

    not_overflowed = signed_azimuthal[~overflow_mask]
    not_overflow_weights = signed_weights[~overflow_mask]

    shifted_plus_overflow = plus_overflow - (2 * np.pi)
    shifted_minus_overflow = minus_overflow + (2 * np.pi)

    signed_azimuthal = ak.concatenate([not_overflowed, shifted_plus_overflow, shifted_minus_overflow], mergebool=True, highlevel=True)
    signed_weights = ak.concatenate([not_overflow_weights, plus_overflow_weights, minus_overflow_weights], mergebool=True, highlevel=True)

    return signed_azimuthal, signed_weights

def scaled_cross_section(weights1, number):
    luminosity = 139 # fb^-1
    normalisation = luminosity/number
    return weights1 * normalisation

def chi_squared(array1, errors1):
    return np.sum((array1/errors1)**2)

def get_system(array, system, *args):
    electrons, muons, jets, missing_et, final_weights = apply_cuts(array)

    dilepton_mass = get_dilepton_mass(electrons, muons)
    dijet_mass = get_dijet_mass(jets)
    transverse_mass = get_transverse_mass(electrons, muons, missing_et)
    signed_azimuthal, signed_azimuthal_weights = get_signed_azimuthal(jets, final_weights)
    signed_azimuthal_l, signed_azimuthal_weights_l = get_signed_azimuthal(electrons, final_weights, muons)
   
    if system =="dilepton mass":
        if len(args)>0:
            ordered_final_jets, ordered_final_weights = order_signed_azimuthal(electrons, final_weights, muons, jets)
            ordered_final_jets_2 = order_signed_azimuthal_overflow(signed_azimuthal_l, ordered_final_jets)
            dijet_mass = get_dijet_mass(ordered_final_jets_2)
            return dijet_mass, ordered_final_weights
        else:
            return dilepton_mass, final_weights
    elif system =="dijet mass":
        if len(args)>0:
            ordered_final_jets, ordered_final_weights = order_signed_azimuthal(jets, final_weights)
            ordered_final_jets_2 = order_signed_azimuthal_overflow(signed_azimuthal, ordered_final_jets)
            dijet_mass = get_dijet_mass(ordered_final_jets_2)
            return dijet_mass, ordered_final_weights
        else:
            return dijet_mass, final_weights
    elif system =="transverse mass":
        return transverse_mass, final_weights
    elif system =="signed azimuthal jets":
        signed_azimuthal_2pi, signed_azimuthal_weights_2pi = signed_azimuthal_overflow(signed_azimuthal, signed_azimuthal_weights)
        return signed_azimuthal_2pi, signed_azimuthal_weights_2pi
    elif system =="signed azimuthal leptons":
        signed_azimuthal_l_2pi, signed_azimuthal_weights_l_2pi = signed_azimuthal_overflow(signed_azimuthal_l, signed_azimuthal_weights_l)
        return signed_azimuthal_l_2pi, signed_azimuthal_weights_l_2pi
    
    elif system == "signed azimuthal lj":
        signed_azimuthal_lj, ordered_jets_lj, ordered_weights_lj = get_signed_azimuthal_lj(jets, electrons, muons, final_weights)
        ordered_weights_lj = scaled_cross_section(ordered_weights_lj, 10000)
        ordered_jets_lj_2 = order_signed_azimuthal_overflow(signed_azimuthal_lj, ordered_jets_lj)
        ordered_dijet_mass_lj = get_dijet_mass(ordered_jets_lj_2)
        signed_azimuthal_lj_2pi, signed_azimuthal_weights_lj_2pi = signed_azimuthal_overflow(signed_azimuthal_lj, ordered_weights_lj)
        return signed_azimuthal_lj_2pi, ordered_dijet_mass_lj, signed_azimuthal_weights_lj_2pi
    
    elif system == "highest pt lepton":
        if len(args)>0:
            highest_lepton_pt, highest_lepton_weights = order_lepton_pt(electrons, muons, final_weights)
            ordered_lepton_pt, ordered_lepton_weights = order_signed_azimuthal(electrons, final_weights, muons, highest_lepton_pt)
            return ordered_lepton_pt, ordered_lepton_weights
        else:
            highest_lepton_pt, highest_lepton_weights = order_lepton_pt(electrons, muons, final_weights)
            return highest_lepton_pt, highest_lepton_weights
         
def plot_histograms(array, weights1, range1, labels):
    array = ak.to_numpy(array)
    weights1 = ak.to_numpy(ak.flatten(weights1))

    nbins = 6
    plt.figure()
    bin_height1 = plt.hist(array, bins=nbins, range=range1, weights=weights1, histtype= 'step', color='b', stacked=True)
    
    plt.title(labels[0])
    plt.xlabel(labels[1])
    plt.ylabel(labels[2])
    plt.close()
    
    
    bin_width = bin_height1[1][1] - bin_height1[1][0]
    bin_centres = 0.5*(bin_height1[1][1:] + bin_height1[1][:-1])
    
    error1 = stats.binned_statistic(array, weights1/bin_width, statistic=variance, bins=nbins, range=range1)

    plt.figure()
    bin_height2 = plt.hist(array, bins=nbins, range=range1, weights=weights1/bin_width, histtype= 'step', color='b', stacked=True)
    bin_centres = 0.5*(bin_height2[1][1:] + bin_height2[1][:-1])
    plt.errorbar(bin_centres, bin_height2[0], yerr=error1[0], fmt='none', color='midnightblue')
    plt.title(labels[0])
    plt.xlabel(labels[1])
    plt.ylabel(labels[2])
    plt.show()

def variance(values):
    return np.sqrt(np.sum(values**2))

def combine_histograms(array1, weights1, label1, array2, weights2, label2, range1, type1, *args):

    array1 = ak.to_numpy(array1)
    weights1 = ak.to_numpy(ak.flatten(weights1))
    array2 = ak.to_numpy(array2)
    weights2 = ak.to_numpy(ak.flatten(weights2))

    if len(type1)>3:
        scale_factor = type1[3]
    else:
        scale_factor = 1

    nbins = 6
    plt.figure()
    bin_height1 = plt.hist(array1, bins=nbins, range=range1, weights=weights1/scale_factor, histtype= 'step', color='mediumblue', label=label1, stacked=True)
    plt.hist(array2, bins=nbins, range=range1, weights=weights2, histtype= 'step', color='red', label=label2, stacked=True)
    plt.title(type1[0])
    plt.xlabel(type1[1])
    plt.ylabel(type1[2])
    plt.legend()
    plt.close()
    
   
    if len(args)>0:
        colours = args[0]
        
    else:
        colours = ('mediumblue', 'red')

    print(colours)
    
    bin_width = bin_height1[1][1] - bin_height1[1][0]
    bin_centres = 0.5*(bin_height1[1][1:] + bin_height1[1][:-1])
    
    error1 = stats.binned_statistic(array1, weights1/bin_width, statistic=variance, bins=nbins, range=range1)
    error2 = stats.binned_statistic(array2, weights2/bin_width, statistic=variance, bins=nbins, range=range1)

    plt.figure()
    bin_height2 = plt.hist(array1, bins=nbins, range=range1, weights=weights1/(bin_width*scale_factor), histtype= 'step', color=colours[0], label=label1, stacked=True)
    bin_centres = 0.5*(bin_height2[1][1:] + bin_height2[1][:-1])
    plt.errorbar(bin_centres, bin_height2[0], yerr=error1[0]/scale_factor, fmt='none', color=colours[0])
    bin_height = plt.hist(array2, bins=nbins, range=range1, weights=weights2/bin_width, histtype= 'step', color=colours[1], label=label2, stacked=True)
    bin_centres = 0.5*(bin_height[1][1:] + bin_height[1][:-1])
    plt.errorbar(bin_centres, bin_height[0], yerr=error2[0], fmt='none', color=colours[1])
    plt.title(type1[0], fontsize = 14)
    plt.xlabel(type1[1], fontsize = 14)
    plt.ylabel(type1[2], fontsize = 14)
    plt.xticks(fontsize = 12)
    plt.yticks(fontsize = 12)
    plt.legend(frameon=False, fontsize = 12)
    #plt.savefig('{}.png'.format(type1[0]), dpi = 1000)
    plt.show()
    print("chi-squared", chi_squared(bin_height[0], error2[0]))

def plot_2d_histogram(array1, array2, weights1, range1, labels):
    array1 = ak.to_numpy(array1)
    array2 = ak.to_numpy(array2)
    weights1 = ak.to_numpy(weights1)

    x_bin_edges = np.linspace(range1[0][0], range1[0][1], 7)
    y_bin_edges = np.arange(range1[1][0], range1[1][1], 0.5)

    bin_edges = [x_bin_edges, y_bin_edges]
    
    nbins = 18

    plt.figure()
    bin_data1 = plt.hist(array1, bins=x_bin_edges, range=range1[0], weights=weights1, histtype='step', color='b')
    bin_data2 = plt.hist(array2, bins=y_bin_edges, range=range1[1], weights=weights1, histtype='step', color='r')
    plt.close()

    bin_width1 = bin_data1[1][1] - bin_data1[1][0]
    bin_width2 = bin_data2[1][1] - bin_data2[1][0]

    plt.figure()
    plt.hist2d(array1, array2, bins= bin_edges, weights=weights1/(bin_width1*bin_width2), range=range1, cmap = 'bwr')
    plt.title(labels[0], fontsize = 14)
    plt.xlabel(labels[1], fontsize = 14)
    plt.ylabel(labels[2], fontsize = 14)
    plt.xticks(fontsize = 12)
    plt.yticks(fontsize = 12)
    plt.colorbar(label = '$d^{2}\sigma/dm d\phi$ [fb/GeV/rad]')
    plt.show()

def plot_split_int_array(array, weights, range1, labels):
    weights_mask_positive = weights > 0
    weights_mask_negative = weights < 0

    positive_weights = weights[weights_mask_positive]
    negative_weights = weights[weights_mask_negative]

    weights_flattened = ak.flatten(weights)

    positive_array = array[weights_flattened > 0]
    negative_array = array[weights_flattened < 0]

    combine_histograms(positive_array, positive_weights, 'Positive', negative_array, negative_weights, 'Negative', range1, labels, ('darkviolet', 'teal'))

def main():
    #SM system

    dilepton_mass_NP0, final_weights_NP0 = get_system(events_NP0, "dilepton mass")
    dijet_mass_NP0, final_weights_NP0 = get_system(events_NP0, "dijet mass")
    transverse_mass_NP0, final_weights_NP0 = get_system(events_NP0, "transverse mass")
    signed_azimuthal_NP0, signed_azimuthal_weights_NP0 = get_system(events_NP0, "signed azimuthal jets")
    signed_azimuthal_lep_NP0, signed_azimuthal_weights_lep_NP0 = get_system(events_NP0, "signed azimuthal leptons")
    ordered_lepton_pt_NP0, ordered_final_weights_NP0 = get_system(events_NP0, "highest pt lepton", "ordered")
    signed_azimuthal_lj_NP0, ordered_jets_lj_NP0, signed_azimuthal_weights_lj_NP0 = get_system(events_NP0, "signed azimuthal lj")
    

    final_weights_NP0 = scaled_cross_section(final_weights_NP0, N_EVENTS_NP0)
    signed_azimuthal_weights_NP0 = scaled_cross_section(signed_azimuthal_weights_NP0, N_EVENTS_NP0)
    signed_azimuthal_weights_lep_NP0 = scaled_cross_section(signed_azimuthal_weights_lep_NP0, N_EVENTS_NP0)
    #NP1 system   

    dilepton_mass_NP1, final_weights_NP1 = get_system(events_NP1, "dilepton mass")
    dijet_mass_NP1, final_weights_NP1 = get_system(events_NP1, "dijet mass")
    transverse_mass_NP1, final_weights_NP1 = get_system(events_NP1, "transverse mass")

    signed_azimuthal_NP1, signed_azimuthal_weights_NP1 = get_system(events_NP1, "signed azimuthal jets")
    ordered_dijet_mass_NP1, ordered_final_weights_NP1 = get_system(events_NP1, "dijet mass", "ordered")
    signed_azimuthal_lep_NP1, signed_azimuthal_weights_lep_NP1 = get_system(events_NP1, "signed azimuthal leptons")
    ordered_dilepton_mass_NP1, ordered_final_weights_NP1 = get_system(events_NP1, "dilepton mass", "ordered")
    ordered_lepton_pt_NP1, ordered_final_weights_NP1 = get_system(events_NP1, "highest pt lepton", "ordered")
    signed_azimuthal_lj_NP1, ordered_jets_lj_NP1, signed_azimuthal_weights_lj_NP1 = get_system(events_NP1, "signed azimuthal lj")
    print(len(signed_azimuthal_lj_NP1))

    final_weights_NP1 = scaled_cross_section(final_weights_NP1, N_EVENTS_NP1)
    signed_azimuthal_weights_NP1 = scaled_cross_section(signed_azimuthal_weights_NP1, N_EVENTS_NP1)
    signed_azimuthal_weights_lep_NP1 = scaled_cross_section(signed_azimuthal_weights_lep_NP1, N_EVENTS_NP1)
    ordered_final_weights_NP1 = scaled_cross_section(ordered_final_weights_NP1, N_EVENTS_NP1)
    #NP2 system  

    dilepton_mass_NP2, final_weights_NP2 = get_system(events_NP2, "dilepton mass")
    dijet_mass_NP2, final_weights_NP2 = get_system(events_NP2, "dijet mass")
    transverse_mass_NP2, final_weights_NP2 = get_system(events_NP2, "transverse mass")
    signed_azimuthal_NP2, signed_azimuthal_weights_NP2 = get_system(events_NP2, "signed azimuthal jets")
    signed_azimuthal_lep_NP2, signed_azimuthal_weights_lep_NP2 = get_system(events_NP2, "signed azimuthal leptons")

    final_weights_NP2 = scaled_cross_section(final_weights_NP2, N_EVENTS_NP2)
    signed_azimuthal_weights_NP2 = scaled_cross_section(signed_azimuthal_weights_NP2, N_EVENTS_NP2)

    #combined arrays

    weights_combined = combine_arrays(final_weights_NP0, final_weights_NP1, final_weights_NP2)
   
    signed_azimuthal_combined = ak.concatenate([signed_azimuthal_NP0, signed_azimuthal_NP1], mergebool=True, highlevel=True)
    signed_azimuthal_weights_combined = ak.concatenate([signed_azimuthal_weights_NP0, signed_azimuthal_weights_NP1], mergebool=True, highlevel=True)
    

    signed_azimuthal_eft_only = ak.concatenate([signed_azimuthal_NP1, signed_azimuthal_NP2], mergebool=True, highlevel=True)
    signed_azimuthal_weights_eft_only = ak.concatenate([signed_azimuthal_weights_NP1, signed_azimuthal_weights_NP2], mergebool=True, highlevel=True)
    signed_azimuthal_lep_combined = ak.concatenate([signed_azimuthal_lep_NP0, signed_azimuthal_lep_NP1], mergebool=True, highlevel=True)
    signed_azimuthal_weights_lep_combined = ak.concatenate([signed_azimuthal_weights_lep_NP0, signed_azimuthal_weights_lep_NP1], mergebool=True, highlevel=True)

    #plot single histograms
    '''
    plot_histograms(signed_azimuthal_NP1, signed_azimuthal_weights_NP1, (-3, 3), ['$\Delta \phi_{jj}$ for VBS region', '$\Delta \phi_{jj}$', '$d\sigma/ d\Delta \phi_{jj}$ [fb/rad]'])
    plot_histograms(signed_azimuthal_lep_NP1, signed_azimuthal_weights_lep_NP1, (-3,3), ['$\Delta \phi_{ll}$ for VBS region', '$\Delta \phi_{ll}$', '$d\sigma/ d\Delta \phi_{ll}$ [fb/rad]'])
    #plot_histograms(ordered_lepton_pt_NP1, ordered_final_weights_NP1, (0, 1000), ['Highest Lepton $p_t$', '$p_t$ [GeV]', 'Differential cross-section'])
    plot_histograms(signed_azimuthal_lj_NP1, signed_azimuthal_weights_lj_NP1, (-3,3), ['$\Delta \phi_{lj}$ for VBS region', '$\Delta \phi_{lj}$', '$d\sigma/ d\Delta \phi_{lj}$ [fb/rad]'])

    combine_histograms(signed_azimuthal_combined, signed_azimuthal_weights_combined, "combined",
                       signed_azimuthal_NP0, signed_azimuthal_weights_NP0, "SM", (-3, 3), ['$\Delta \phi_{jj}$ for $160<m_{jj}<700$ GeV', '$\Delta \phi_{jj}$', '$d\sigma/ d\Delta \phi_{jj}$ [fb/rad]'])
    combine_histograms(signed_azimuthal_lep_combined, signed_azimuthal_weights_lep_combined, "combined",
                          signed_azimuthal_lep_NP0, signed_azimuthal_weights_lep_NP0, "SM", (-3, 3), ['$\Delta \phi_{ll}$ for $160<m_{jj}<700$ GeV', '$\Delta \phi_{ll}$', '$d\sigma/ d\Delta \phi_{ll}$ [fb/rad]'])
    #histograms showing observables for different NP
    
    combine_histograms(dilepton_mass_NP0, final_weights_NP0/5, "SM",
                          dilepton_mass_NP1, final_weights_NP1, "Int",
                          (0, 500), ['Dilepton mass', '$m_{jj}$ [GeV]', 'Differential cross-section'])
    '''
    scale_factor = 5

    coefficient_label = r"$\mathcal{\tilde{c}}_{ \widetilde{W}}/\Lambda^2 = 1 \mathrm{TeV}^{-2}$"
    combine_histograms(signed_azimuthal_NP0, signed_azimuthal_weights_NP0, "SM/{0}".format(scale_factor), 
                       signed_azimuthal_NP1, signed_azimuthal_weights_NP1, coefficient_label,
                       (-3, 3), [r'$\Delta \phi_{jj}$ for VBS region', 
                                 '$\Delta \phi_{jj}$ [rad]', '$d\sigma/ d\Delta \phi_{jj}$ [fb/rad]', scale_factor])
                       #signed_azimuthal_NP2, signed_azimuthal_weights_NP2)

    plot_split_int_array(signed_azimuthal_NP1, signed_azimuthal_weights_NP1, (-3, 3), 
                         [r'$\Delta \phi_{jj}$ for VBS region, $\mathcal{\tilde{c}}_{ \widetilde{W}}/\Lambda^2 = 1 \mathrm{TeV}^{-2}$', 
                          '$\Delta \phi_{jj}$ [rad]', '$d\sigma/ d\Delta \phi_{jj}$ [fb/rad]'])

    combine_histograms(signed_azimuthal_lep_NP0, signed_azimuthal_weights_lep_NP0, "SM/{0}".format(scale_factor),
                       signed_azimuthal_lep_NP1, signed_azimuthal_weights_lep_NP1, coefficient_label,
                       (-3, 3), [r'$\Delta \phi_{ll}$ for VBS region', 
                                 '$\Delta \phi_{ll}$ [rad]', '$d\sigma/ d\Delta \phi_{ll}$ [fb/rad]', scale_factor])
    
    plot_split_int_array(signed_azimuthal_lep_NP1, signed_azimuthal_weights_lep_NP1, (-3, 3), 
                         [r'$\Delta \phi_{ll}$ for VBS region, $\mathcal{\tilde{c}}_{ \widetilde{W}}/\Lambda^2 = 1 \mathrm{TeV}^{-2}$', 
                          '$\Delta \phi_{ll}$ [rad]', '$d\sigma/ d\Delta \phi_{ll}$ [fb/rad]'])
    
    combine_histograms(signed_azimuthal_lj_NP0, signed_azimuthal_weights_lj_NP0, "SM/{0}".format(scale_factor),
                          signed_azimuthal_lj_NP1, signed_azimuthal_weights_lj_NP1, coefficient_label,
                            (-3, 3), [r'$\Delta \phi_{lj}$ for VBS region', 
                                      '$\Delta \phi_{lj}$ [rad]', '$d\sigma/ d\Delta \phi_{lj}$ [fb/rad]', scale_factor])
    
    plot_split_int_array(signed_azimuthal_lj_NP1, signed_azimuthal_weights_lj_NP1, (-3, 3), 
                         [r'$\Delta \phi_{lj}$ for VBS region, $\mathcal{\tilde{O}}_{ \widetilde{W}} = 1 \mathrm{TeV}^{-2}$', 
                          '$\Delta \phi_{lj}$ [rad]', '$d\sigma/ d\Delta \phi_{lj}$ [fb/rad]'])

    plot_2d_histogram(ordered_dijet_mass_NP1, signed_azimuthal_NP1, ak.flatten(signed_azimuthal_weights_NP1), 
                      [[100,1000], [-3,3]], ['$100 < m_{jj}< 1000$ GeV','$m_{jj}$ [GeV]', '$\Delta \phi_{jj}$ [rad]'])
    plot_2d_histogram(ordered_dilepton_mass_NP1, signed_azimuthal_lep_NP1, ak.flatten(signed_azimuthal_weights_lep_NP1), 
                      [[100, 1000], [-3,3]], ['$100 < m_{jj}< 1000$ GeV','$m_{jj}$ [GeV]', '$\Delta \phi_{ll}$ [rad]'])
    plot_2d_histogram(ordered_jets_lj_NP1, signed_azimuthal_lj_NP1, ak.flatten(signed_azimuthal_weights_lj_NP1), 
                      [[100, 1000], [-3,3]], ['$100 < m_{jj}< 1000$ GeV','$m_{jj}$ [GeV]', '$\Delta \phi_{lj}$ [rad]'])
    print(signed_azimuthal_weights_NP1)
main()